In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.style as style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 140 #set_size
import seaborn as sns
import missingno as msno
import numpy as np
import pandas as pd
from plotly.offline import iplot, init_notebook_mode
import plotly.express as px
import plotly.graph_objs as go
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)
import folium
import geopandas as gpd

Reading the data first list of countries to plot in the map.

In [ ]:

iso_countries = pd.read_csv('/kaggle/input/country-mapping-iso-continent-region/continents2.csv')
iso_countries.head()


Reading the data 

In [ ]:
data = pd.read_csv('/kaggle/input/who-covid19-data-tabe/WHO COVID-19 global table data August 11th 2021 at 10.41.34 AM.csv')
data.head()

Checking the nan values from the data

In [ ]:
msno.matrix(data)

In [ ]:
msno.bar(data)

Drop the column Deaths - newly reporterd in the last 24 hours. It is all nan

In [ ]:
data.pop('Deaths - newly reported in last 24 hours')

Check the size of the data after drop the column

In [ ]:
len(data)


preparing the database to delete rows with all 0 values

In [ ]:

data['Name'] = data['Name'].replace(np.nan, 'All')
data = data.replace(0 , np.NAN)
data = data.dropna(how='any', axis=0)

data

Changing the column name

In [ ]:
data =data.rename(columns =  {'Name': 'Continent'}, inplace = False) 

reset the index 

In [ ]:
data=data.reset_index()


rename the old index column

In [ ]:

data =data.rename(columns =  {'index': 'Country'}, inplace = False) 
data

Rename all the countrys to fit in the data from countries to plot

In [ ]:
data['Country']= data['Country'].replace('United States of America', 'United States')
data['Country']= data['Country'].replace('Russian Federation', 'Russian')
data['Country']= data['Country'].replace('The United Kingdom' , 'United Kingdom')
data['Country']= data['Country'].replace('Iran (Islamic Republic of)' , 'Iran')
data['Country']= data['Country'].replace('Czechia' , 'Czech Republic')
data['Country']= data['Country'].replace('occupied Palestinian territory, including east Jerusalem' , 'Palestine, State of')
data['Country']= data['Country'].replace('Republic of Moldova' , 'Moldova')
data['Country']= data['Country'].replace('Viet Nam' , 'Vietnam')
data['Country']= data['Country'].replace('Bolivia (Plurinational State of)' , 'Bolivia')
data['Country']= data['Country'].replace( 'Palestine','Palestine, State of')
data['Country']= data['Country'].replace('Venezuela (Bolivarian Republic of)' , 'Venezuela')
data['Country']= data['Country'].replace('Republic of Korea' , 'Korea, Republic of')
data['Country']= data['Country'].replace( 'Korea(Republic of','Korea, Republic of')
data['Country']= data['Country'].replace( 'Bosnia and Herzegovina','Bosnia And Herzegovina')
data['Country']= data['Country'].replace( ' Namibia ','Namibia')
data['Country']= data['Country'].replace( 'Syrian Arab Republic','Syria')
data['Country']= data['Country'].replace( 'United Republic of Tanzania','Tanzania')




In [ ]:
data =data.drop([0])

# Show the biggest values in each dimension by heatmap

In [ ]:
data_country = data.sort_values(['Deaths - cumulative total per 100000 population'], ascending = False).reset_index(drop=True)
data_country.drop(columns = ['WHO Region', 'Cases - cumulative total', 'Cases - newly reported in last 7 days']).head(10).style.background_gradient(cmap='Reds', subset = ['Deaths - cumulative total per 100000 population','Deaths - newly reported in last 7 days','Deaths - newly reported in last 7 days per 100000 population','Deaths - cumulative total','Cases - newly reported in last 24 hours','Cases - newly reported in last 7 days per 100000 population','Cases - cumulative total per 100000 population'])

Merge the data to plot the map

In [ ]:
data_plot = data.merge(iso_countries, how='left', left_on=['Country'], right_on=['name'])
data_plot.rename(columns={'Alpha-3 code':'country code','English short name lower case': 'country name'}, inplace=True)
data_plot

# Map showing the countries cumulative total per 100000 population

In [ ]:
fig = px.choropleth(data_plot, locations="alpha-3",color="Deaths - cumulative total per 100000 population",hover_name="Country",color_continuous_scale=px.colors.sequential.YlOrRd)
layout = go.Layout(title=go.layout.Title(text="Deaths - cumulative total per 100000 population",x=0.10),font=dict(size=10),width = 1000,height = 500,margin=dict(l=1,r=1,b=1,t=20))
fig.update_layout(layout)
fig.show()

# Map showhing Deaths - newly reported in last 7 days

In [ ]:
fig = px.choropleth(data_plot, locations="alpha-3",color="Deaths - newly reported in last 7 days",hover_name="Country",color_continuous_scale=px.colors.sequential.YlOrRd)
layout = go.Layout(title=go.layout.Title(text="Deaths - newly reported in last 7 days",x=0.10),font=dict(size=10),width = 1000,height = 500,margin=dict(l=1,r=1,b=1,t=20))
fig.update_layout(layout)
fig.show()